In [ ]:
from init_notebook import *
from src.algo.boulderdash import *

In [ ]:
SIZE = 32
COUNT = 60_000    
PRE_STEPS = 1
PREDICTION_STEPS = 1

In [ ]:
ds_test = BoulderDashIterableDataset(
    count=COUNT, shape=(SIZE, SIZE), pre_steps=PRE_STEPS, prediction_steps=PREDICTION_STEPS, dtype=torch.int8,
)


for map1, map2 in ds_test:
    size_in_bytes = map1.nbytes * 2 * COUNT
    image1, image2 = BoulderDash.from_tensor(map1).to_image(16), BoulderDash.from_tensor(map2).to_image(16)
    break

print(f"size in bytes: {size_in_bytes:,}")
VF.to_pil_image(make_grid([torch.from_numpy(image1), torch.from_numpy(image2)], padding=8))

# store tensors

In [ ]:
filename_part = f"../datasets/boulderdash-{SIZE}x{SIZE}-{COUNT}"

map1_batch = []
map2_batch = []
for map1, map2 in tqdm(ds_test):
    map1_batch.append(map1.unsqueeze(0))
    map2_batch.append(map2.unsqueeze(0))
    #if len(map1_batch) >= 10:
    #    break

map1_batch = torch.concat(map1_batch)
map2_batch = torch.concat(map2_batch)
torch.save(map1_batch, f"{filename_part}-map1.pt")
torch.save(map2_batch, f"{filename_part}-map2.pt")

## test

In [ ]:
ds = TensorDataset(
    torch.load(f"{filename_part}-map1.pt"),
    torch.load(f"{filename_part}-map2.pt"),
)
print(len(ds))
print([i.shape for i in ds[0]])

In [ ]:
from src.transforms import *

def boulderdash_dataset_32x32(
        validation: bool,
):
    path = Path("..") / "datasets"
    if validation:
        filename_part = "boulderdash-32x32-5000-validation"
        return WrapDataset(TensorDataset(
            torch.load(path / f"{filename_part}-map1.pt"),
            torch.load(path / f"{filename_part}-map2.pt"),
        )).transform(dtype=torch.float, transform_all=True)

    filename_part = "boulderdash-64x64-20000"
    ds = WrapDataset(TensorDataset(
        torch.load(path / f"{filename_part}-map1.pt"),
        torch.load(path / f"{filename_part}-map2.pt"),
    ))
    ds = ds.transform([RandomCropTuple(32)], dtype=torch.float, transform_all=True, num_repeat=4)
        
    return ds

for _ in tqdm(boulderdash_dataset_32x32(validation=False)):
    pass

In [ ]:
from experiments.datasets import boulderdash_dataset_32x32
ds = boulderdash_dataset_32x32(validation=False)
map1, map2 = ds[0]
display(VF.to_pil_image(make_grid([
    torch.from_numpy(BoulderDash.from_tensor(map1).to_image()),
    torch.from_numpy(BoulderDash.from_tensor(map2).to_image()),
], padding=8)))
print(map1)
print(map2)